In [2]:
import numpy as np

In [10]:
batch_size = 10**4
num_concepts = 10
num_latent_concepts = 5

In [165]:
# sampling from independent normals using parameterization trick
def sample(num_concepts, num_latent_concepts, batch_size):
    mu_hat = np.random.random(num_concepts).reshape(num_concepts, 1) # (n_concepts,  1)
    true_sigma_hat = np.random.random(num_concepts).reshape(num_concepts, 1) # (n_concepts,  1)
    c_hat = np.random.standard_normal(batch_size*num_concepts).reshape(batch_size, num_concepts, 1) * true_sigma_hat**0.5 + mu_hat # (batch_size, n_concepts,  1)

    mu_tilde = np.random.random(num_latent_concepts).reshape(num_latent_concepts, 1) # (n_latent,  1)
    true_sigma_tilde = np.random.random(num_latent_concepts).reshape(num_latent_concepts, 1) # (n_latent,  1)
    c_tilde = np.random.standard_normal(batch_size*num_latent_concepts).reshape(batch_size, num_latent_concepts, 1) * true_sigma_tilde**0.5 + mu_tilde # (batch_size, n_latent,  1)
    # print(list(true_sigma_hat.squeeze())+list(true_sigma_tilde.squeeze()))
    return c_hat, c_tilde

# c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
# c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
# print(c_hat.shape, c_tilde.shape)

In [171]:
batch_size = 12
num_concepts = 10
num_latent_concepts = 5

In [212]:
def covariance_short(c_hat, c_tilde):
    c_hat = c_hat.squeeze(axis=-1).T
    c_tilde = c_tilde.squeeze(axis=-1).T
    samples = np.concatenate((c_hat, c_tilde), axis=0)
    
    samples_mean = np.mean(samples, axis=1)
    exey = np.outer(samples_mean, samples_mean)
    exy = np.zeros(exey.shape)
    cov_again = np.zeros(exey.shape)
    for i in range(exey.shape[0]):
        mean_row_i = np.mean(samples[i])
        sample_mean_row_i = np.sum(samples[i]) / (len(samples[i])-1)
        for j in range(exey.shape[1]):
            mean_row_j = np.mean(samples[j])
            sample_mean_row_j = np.sum(samples[j]) / (len(samples[j])-1)
            # cov_again[i,j] = np.mean(samples[i]*samples[j]) - mean_row_i*mean_row_j
            cov_again[i,j] = np.sum((samples[i]-mean_row_i) * (samples[j]-mean_row_j)) / (len(samples[j])-1)
            # cov_again[i,j] = np.sum( samples[i]*samples[j] - samples[i]*mean_row_j - mean_row_i*samples[j] + mean_row_i*mean_row_j ) / (len(samples[j])-1)
            # cov_again[i,j] = np.sum( samples[i]*samples[j] ) / (len(samples[j])-1) - sample_mean_row_i*mean_row_j - mean_row_i*sample_mean_row_j + mean_row_i*mean_row_j
            # cov_again[i,j] = np.sum(samples[i]*samples[j])/(samples.shape[1]) - mean_row_i*mean_row_j
            exy[i,j] = np.mean(samples[i]*samples[j])
    # print(exy.shape)
    # exy_true = np.cov(samples) + exey

    # print(exy)
    # print()
    # print(exy_true)


    cov = exy - exey
    return cov_again

def covariance_np(c_hat, c_tilde):
    c_hat = c_hat.squeeze(axis=-1).T
    c_tilde = c_tilde.squeeze(axis=-1).T
    samples = np.concatenate((c_hat, c_tilde), axis=0)
    
    return np.cov(samples)


    # print(samples_mean.shape)
    # cov = 

c_hat, c_tilde = sample(num_concepts, num_latent_concepts, batch_size)
sigma = covariance(c_hat, c_tilde)
sigma_np = covariance_np(c_hat, c_tilde)
print(sigma)
print()
print(sigma_np)

[[ 0.52781254 -0.00342716 -0.13900249  0.22957528]
 [-0.00342716  0.07193949 -0.05867424  0.10551622]
 [-0.13900249 -0.05867424  0.4839676  -0.02610297]
 [ 0.22957528  0.10551622 -0.02610297  0.70553226]]

[[ 0.52781254 -0.00342716 -0.13900249  0.22957528]
 [-0.00342716  0.07193949 -0.05867424  0.10551622]
 [-0.13900249 -0.05867424  0.4839676  -0.02610297]
 [ 0.22957528  0.10551622 -0.02610297  0.70553226]]


In [211]:
def covariance(c_hat, c_tilde):
    c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
    c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
    c_stack_T = np.concatenate((c_hat,c_tilde), axis=1) # (batch_size, n_concepts+n_latent, 1)
    c_hat_mean_resize = np.repeat(c_hat_mean.reshape(1,num_concepts,1), batch_size, axis=0) # (batch_size, n_concepts, 1)
    c_tilde_mean_resize = np.repeat(c_tilde_mean.reshape(1,num_latent_concepts,1), batch_size, axis=0) # (batch_size, n_latent, 1)
    c_stack_mean_T = np.concatenate((c_hat_mean_resize,c_tilde_mean_resize), axis=1)  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation = c_stack_T-c_stack_mean_T  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation_T = np.moveaxis(c_stack_deviation,-1,-2)  # (batch_size, 1, n_concepts+n_latent)
    sigma = 1/(batch_size-1) * np.sum(np.matmul(c_stack_deviation,c_stack_deviation_T),axis=0)  # (batch_size, n_concepts+n_latent, n_concepts+n_latent)
    return sigma

In [111]:
c_hat, c_tilde = sample(num_concepts, num_latent_concepts, batch_size)
sigma_scratch = covariance(c_hat, c_tilde)
c_hat = c_hat.squeeze(axis=-1).T
c_tilde = c_tilde.squeeze(axis=-1).T
samples = np.concatenate((c_hat, c_tilde), axis=0)
sigma_np = np.cov(samples)
print(sigma_np)
print()
print(sigma_scratch)
print(np.sum(np.abs(sigma_scratch - sigma_np)))
print(np.linalg.det(sigma_np), np.linalg.det(sigma_scratch))

[0.8135731125215057, 0.7576167103399972, 0.31160523050571065, 0.8604086080584533]
[[ 0.67459205 -0.415014    0.00300902 -0.02950988]
 [-0.415014    1.25191919 -0.17795501 -0.46840892]
 [ 0.00300902 -0.17795501  0.27022026  0.0027806 ]
 [-0.02950988 -0.46840892  0.0027806   0.86797543]]

[[ 0.60713285 -0.3735126   0.00270811 -0.02655889]
 [-0.3735126   1.12672727 -0.16015951 -0.42156803]
 [ 0.00270811 -0.16015951  0.24319824  0.00250254]
 [-0.02655889 -0.42156803  0.00250254  0.78117789]]
0.5258061794321258
0.09648715386070177 0.06330522164800642


In [207]:
def loss_np(c_hat, c_tilde):
    c_hat = c_hat.squeeze(axis=-1).T
    c_tilde = c_tilde.squeeze(axis=-1).T
    samples = np.concatenate((c_hat, c_tilde), axis=0)
    sigma = np.cov(samples)
    # print(sigma)
    det_sigma = np.linalg.det(sigma)
    # print(det_sigma)
    log_det_sigma = np.log(det_sigma)

    # print(samples[c_hat.shape[0]:, c_hat.shape[0]:].shape)
    sigma_tilde = np.cov(samples[c_hat.shape[0]:, c_hat.shape[0]:])
    det_sigma_tilde = np.linalg.det(sigma_tilde)
    log_det_sigma_tilde = np.log(det_sigma_tilde) # scalar
    r = (log_det_sigma - log_det_sigma_tilde) / 2
    return r

In [226]:
def covariance(c_hat, c_tilde):
    c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
    c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
    c_stack_T = np.concatenate((c_hat,c_tilde), axis=1) # (batch_size, n_concepts+n_latent, 1)
    c_hat_mean_resize = np.repeat(c_hat_mean.reshape(1,num_concepts,1), batch_size, axis=0) # (batch_size, n_concepts, 1)
    c_tilde_mean_resize = np.repeat(c_tilde_mean.reshape(1,num_latent_concepts,1), batch_size, axis=0) # (batch_size, n_latent, 1)
    c_stack_mean_T = np.concatenate((c_hat_mean_resize,c_tilde_mean_resize), axis=1)  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation = c_stack_T-c_stack_mean_T  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation_T = np.moveaxis(c_stack_deviation,-1,-2)  # (batch_size, 1, n_concepts+n_latent)
    sigma = 1/(batch_size-1) * np.sum(np.matmul(c_stack_deviation,c_stack_deviation_T),axis=0)  # (batch_size, n_concepts+n_latent, n_concepts+n_latent)
    return sigma

def loss(c_hat, c_tilde, covariance):
    # c_hat = c_hat.squeeze(axis=-1).T
    # c_tilde = c_tilde.squeeze(axis=-1).T
    # samples = np.concatenate((c_hat, c_tilde), axis=0)
    sigma = covariance(c_hat, c_tilde)
    # print(sigma)
    det_sigma = np.linalg.det(sigma)
    # print(det_sigma)
    log_det_sigma = np.log(det_sigma)

    # print(samples[c_hat.shape[0]:, c_hat.shape[0]:].shape)
    sigma_tilde = sigma[c_hat.shape[1]:, c_hat.shape[1]:]
    det_sigma_tilde = np.linalg.det(sigma_tilde)
    log_det_sigma_tilde = np.log(det_sigma_tilde) # scalar
    r = (log_det_sigma - log_det_sigma_tilde) / 2
    return r

In [229]:
def loss_term(c_hat, c_tilde):
    c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
    c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)

    c_stack_T = np.concatenate((c_hat,c_tilde), axis=1) # (batch_size, n_concepts+n_latent, 1)
    c_hat_mean_resize = np.repeat(c_hat_mean.reshape(1,num_concepts,1), batch_size, axis=0) # (batch_size, n_concepts, 1)
    c_tilde_mean_resize = np.repeat(c_tilde_mean.reshape(1,num_latent_concepts,1), batch_size, axis=0) # (batch_size, n_latent, 1)
    c_stack_mean_T = np.concatenate((c_hat_mean_resize,c_tilde_mean_resize), axis=1)  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation = c_stack_T-c_stack_mean_T  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation_T = np.moveaxis(c_stack_deviation,-1,-2)  # (batch_size, 1, n_concepts+n_latent)
    sigma = 1/(batch_size-1) * np.sum(np.matmul(c_stack_deviation,c_stack_deviation_T),axis=0)  # (batch_size, n_concepts+n_latent, n_concepts+n_latent)
    # sigma = covariance(c_hat, c_tilde)
    log_det_sigma = np.log(np.linalg.det(sigma)) # scalar

    c_tilde_deviation = c_tilde-c_tilde_mean # (batch_size, n_latent, 1)
    c_tilde_deviation_T = np.moveaxis(c_tilde_deviation,-1,-2) # (batch_size, 1, n_latent)
    sigma_tilde = 1/(batch_size-1) * np.sum(np.matmul(c_tilde_deviation, c_tilde_deviation_T),axis=0)  # (n_latent, n_latent)
    # sigma_tilde = sigma[c_hat.shape[1]:, c_hat.shape[1]:]
    log_det_sigma_tilde = np.log(np.linalg.det(sigma_tilde)) # scalar

    r = (log_det_sigma - log_det_sigma_tilde) / 2
    return r

In [232]:
batch_size = 10
num_concepts = 7
num_latent_concepts = 4

for _ in range(100):
    c_hat, c_tilde = sample(num_concepts, num_latent_concepts, batch_size)
    # r = loss_term(c_hat, c_tilde)
    # r_fast = loss_np(c_hat, c_tilde)
    r = loss(c_hat, c_tilde, covariance_np)
    # r_fast = loss(c_hat, c_tilde, covariance)
    r_fast = loss_term(c_hat, c_tilde)
    # print(r_fast)
    # if np.isnan(r_fast):
    #     break
    print(r, r_fast)
    # print(r)

-41.084284197087044 -40.9086522377501
-42.137530048934295 nan
nan -40.39988889182447
nan nan
-inf nan
-44.02085428672016 -43.56064762977157
-40.58434335170526 nan
-41.505314983439085 nan
-46.32256784863933 -45.229031745703224
-40.56238790110159 -41.06579027080908
nan nan
-43.68012419514183 nan
nan nan
-inf nan
-42.533654837691 -42.5277376088675
-43.05203536061653 nan
-42.88556288283602 -42.68247294166345
-45.19056968665869 nan
-46.35851228733922 nan
-44.730288835756056 -43.952192136798416
nan nan
nan nan
-44.46914255509125 -45.080475023760314
nan nan
nan nan
-41.96386476976567 -42.66650924606349
nan -43.53196394342383
nan nan
-44.49876164274746 nan
nan nan
nan nan
-41.373704241657286 nan
nan -45.43108380180514
nan -41.68178148305259
nan -44.08505439149189
nan nan
nan nan
nan nan
-42.3491365378795 nan
nan -41.5042444151433
nan nan
nan nan
nan nan
-43.83986429296105 -43.38866967164441
-43.85107627150666 -44.29383580811036
-inf nan
nan -41.59788245355249
-40.98504187398684 -inf
-39.775143

/tmp/ipykernel_1040/3403182430.py:13: RuntimeWarning: invalid value encountered in log
  log_det_sigma = np.log(np.linalg.det(sigma)) # scalar
/tmp/ipykernel_1040/3746537734.py:21: RuntimeWarning: invalid value encountered in log
  log_det_sigma = np.log(det_sigma)
/tmp/ipykernel_1040/3746537734.py:21: RuntimeWarning: divide by zero encountered in log
  log_det_sigma = np.log(det_sigma)
/tmp/ipykernel_1040/3403182430.py:13: RuntimeWarning: divide by zero encountered in log
  log_det_sigma = np.log(np.linalg.det(sigma)) # scalar


In [115]:
print(r)

0.5875920078389916
